In [14]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from keras.layers import LSTM, Bidirectional
from keras.layers import SpatialDropout1D, Dropout, Conv1D, Flatten
from keras.layers import GlobalAveragePooling1D,GlobalMaxPool1D, MaxPooling1D, AveragePooling1D
from keras.optimizers import Adam, Adamax, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

import re
import string
import nltk
from nltk.corpus import stopwords

from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

In [15]:
dirdata = 'data/'

In [16]:
#Train data
train_df = pd.read_csv(dirdata + 'train_fttd.csv')
train_df.head()


,id,keyword,location,text,target,keyword_clean,location_clean,text_clean
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,,NaN,our deeds are the reason of this earthquake ma...
1,4,,,Forest fire near La Ronge Sask. Canada,1,,NaN,forest fire near la ronge sask canada
2,5,,,All residents asked to 'shelter in place' are ...,1,,NaN,all residents asked to shelter in place are be...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,,NaN,13000 people receive wildfires evacuation orde...
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,,NaN,just got sent this photo from ruby alaska as s...


In [17]:
#Test data 
test_df = pd.read_csv(dirdata + 'test_fttd.csv')
test_df.head()

,id,keyword,location,text,keyword_clean,location_clean,text_clean
0,0,,,Just happened a terrible car crash,,NaN,just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s...",,NaN,heard about earthquake is different cities sta...
2,3,,,"there is a forest fire at spot pond, geese are...",,NaN,there is a forest fire at spot pond geese are ...
3,9,,,Apocalypse lighting. #Spokane #wildfires,,NaN,apocalypse lighting spokane wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,,NaN,typhoon hanna kills 28 in china and taiwan


In [18]:
#El valor máximo es 34, probé con 32 y 64. No usé más para no agregar ruido.
MAX_LEN = 64
BATCH_SIZE = 16
EPOCHS = 3
LR = 1e-5

# Funciones(Tokenizar, encodear, etc)

In [19]:
def resultados(pred, test_df, algoritmo):
    res_df=pd.DataFrame(test_df['id'])
    res_df['target']=pred
    res_df.to_csv('data/submission' + algoritmo + '.csv', index=False)

In [20]:

def bert_encode(texts, tokenizer, max_len = 128):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [21]:
#Agragando más capas tarda muuuh o cambio de nada casi.
def build_model(bert_layer, max_len = 128):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = clf_output
    print('Shape: ',clf_output.shape)
    x = Dense(1, activation='sigmoid')(x)
    #optimizer = SGD(learning_rate=1e-5)
    optimizer = Adamax(learning_rate=LR)
    #optimizer = Adam(learning_rate=LR)
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=x)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])   
    return model

In [22]:
'''
# Intente agregar más capas pero el modelo empieza a empeorar, overtittea.
x = Flatten()(x)
x = Dense(256, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense (256, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = 'sigmoid')(x)
'''

"\n# Intente agregar más capas pero el modelo empieza a empeorar, overtittea.\nx = Flatten()(x)\nx = Dense(256, activation = 'relu')(x)\nx = Dropout(0.5)(x)\nx = Dense (256, activation = 'relu')(x)\nx = Dropout(0.5)(x)\nx = Dense(1, activation = 'sigmoid')(x)\n"

# BERT

In [ ]:
#Se probaron varios Embbedings pero algunos no se terminaron debido al excesivo consumo de memoria. Con 5 apochs y 10 de batch_size anda bien(37 min).
url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1' # Mas lento pero mejor..
#url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
#url = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2'
#url = 'https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/2'
bert_layer = hub.KerasLayer(url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
print('do_lower_case:', do_lower_case)

In [ ]:
train_input = bert_encode(train_df.text_clean.values, tokenizer, max_len=MAX_LEN)
test_input = bert_encode(test_df.text_clean.values, tokenizer, max_len=MAX_LEN)
train_labels = train_df.target.values
#train_labels = pd.get_dummies(train_df.target).values

In [ ]:
model = build_model(bert_layer, max_len=MAX_LEN)
model.summary()

In [ ]:
# Usando la GPU menos 20 de batch_size anda bien. Aumentando a 20 nos quedamos sin memoria. Otra opción es aumentar los Epochs.
# Con 3 anda bien. Probé hasta 50 pero no se consigue buenos resultados. Converge rápidamente.

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=EPOCHS,
    callbacks=[checkpoint],
    batch_size=BATCH_SIZE
)

> 

# Salida

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)
#submission_df['target'] = np.argmax(test_pred, axis=1)
y_pred = np.round(test_pred).astype(int)

#Salida
resultados(y_pred, test_df, 'BERT-LOCAL')